# Advanced Tutorial on metasyn

In this tutorial, we will be creating synthetic data using the `metasyn` package.

Some advanced features of metasyn will be covered further along the tutorial, such as handling dates, setting distributions, ensuring uniqueness in columns and adding variable descriptions.

For more information refer to the [user's guide](https://metasynth.readthedocs.io/en/latest/usage/usage.html) on the docs.

## Step 0: Install the metasyn package and import required packages

First, let's install the metasyn package.

In [7]:
# Run the following line to install metasyn
# %pip install metasyn

Now, let's import the required packages.

In [8]:
# import required packages
import polars as pl
from metasyn import MetaFrame, demo_file
from metasyn.config import VarConfig
from metasyn.util import DistributionSpec

## Step 1: Loading the dataset

The first step to create synthetic data is to load your dataset into a DataFrame. For this tutorial, we will be using the [Titanic dataset](https://www.kaggle.com/c/titanic/data), which can easily be accessed through the metasyn `demo_file()` function. 

It is important to set the data types of columns in the DataFrame correctly, as this will help metasyn to infer the correct distributions for each variable later.


> **Note** 
> In this tutorial we use [Polars](https://pola.rs) to create the DataFrame, as that is what metasyn uses internally. Pandas is also supported, but will automatically be converted to Polars by metasyn. For best results it is recommended to use Polars.

In [9]:
# get the demonstration data file
csv_path = demo_file("titanic")

# ensure columns are of the correct type
data_types = {"Sex": pl.Categorical, "Embarked": pl.Categorical}

# read the data from the csv path
df = pl.read_csv(csv_path, dtypes=data_types, try_parse_dates=True)

# check out the data
df.head()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],str
1,"""Braund, Mr. Ow…","""male""",22,0,"""A/5 21171""",7.25,null,"""S""",1937-10-28,15:53:04,2022-08-05 04:43:34,null
2,"""Cumings, Mrs. …","""female""",38,0,"""PC 17599""",71.2833,"""C85""","""C""",null,12:26:00,2022-08-07 01:56:33,null
3,"""Heikkinen, Mis…","""female""",26,0,"""STON/O2. 31012…",7.925,null,"""S""",1931-09-24,16:08:25,2022-08-04 20:27:37,null
4,"""Futrelle, Mrs.…","""female""",35,0,"""113803""",53.1,"""C123""","""S""",1936-11-30,null,2022-08-07 07:05:55,null
5,"""Allen, Mr. Wil…","""male""",35,0,"""373450""",8.05,null,"""S""",1918-11-07,10:59:08,2022-08-02 15:13:34,null


We can check the data types of our DataFrame as follows:

In [10]:
dict(zip(df.columns, df.dtypes))

{'PassengerId': Int64,
 'Name': Utf8,
 'Sex': Categorical,
 'Age': Int64,
 'Parch': Int64,
 'Ticket': Utf8,
 'Fare': Float64,
 'Cabin': Utf8,
 'Embarked': Categorical,
 'Birthday': Date,
 'Board time': Time,
 'Married since': Datetime(time_unit='us', time_zone=None),
 'all_NA': Utf8}

We see that most variables are now nicely specified as strings, categories, dates and ints where necessary. We can get some more information on the DataFrame by calling the `describe()` on it, this will give us some information on the distribution of the variables:  

In [11]:
df.describe()

describe,PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
str,f64,str,str,f64,f64,str,f64,str,str,str,str,str,str
"""count""",891.0,"""891""","""891""",891.0,891.0,"""891""",891.0,"""891""","""891""","""891""","""891""","""891""","""891"""
"""null_count""",0.0,"""0""","""0""",177.0,0.0,"""0""",0.0,"""687""","""2""","""78""","""79""","""92""","""891"""
"""mean""",446.0,null,null,29.693277,0.381594,null,32.204208,null,null,null,null,null,null
"""std""",257.353842,null,null,14.524527,0.806057,null,49.693429,null,null,null,null,null,null
"""min""",1.0,"""Abbing, Mr. An…",null,0.0,0.0,"""110152""",0.0,"""A10""",null,"""1903-07-28""","""10:39:40""","""2022-07-15 12:…",null
"""25%""",223.0,null,null,20.0,0.0,null,7.8958,null,null,null,null,null,null
"""50%""",446.0,null,null,28.0,0.0,null,14.4542,null,null,null,null,null,null
"""75%""",669.0,null,null,38.0,0.0,null,31.0,null,null,null,null,null,null
"""max""",891.0,"""van Melkebeke,…",null,80.0,6.0,"""WE/P 5735""",512.3292,"""T""",null,"""1940-05-27""","""18:39:28""","""2022-08-15 10:…",null


## Step 2: Generating a MetaFrame

Now that we have properly formatted our DataFrame, we can easily generate a MetaFrame for it. 
We'll do this without passing in any optional parameters, but later on in this tutorial we will cover how custom parameters can help provide control over the MetaFrame generation process. 

> **MetaFrames:**
> A MetaFrame is an object which captures the essential aspects of the dataset, including variable names, types, data types, the percentage of missing values, and distribution parameters. MetaFrame objects capture all the information needed to generate a synthetic dataset that aligns with the original dataset, without containing any *entries* of the original dataset.

More information on generating MetaFrames can be found on the metasyn docs ['generating metaframes'](https://metasynth.readthedocs.io/en/latest/usage/generating_metaframes.html) page.

Generating a MetaFrame is simple, and can be done by simply calling the `MetaFrame.fit_dataframe()` class method, passing in the DataFrame as a parameter.

In [12]:
# Generate and fit a MetaFrame to the DataFrame 
mf = MetaFrame.fit_dataframe(df)

100%|██████████| 13/13 [00:13<00:00,  1.01s/it]


We can use the built-in Python `print` function to display the (statistical) metadata contained in the MetaFrame in an easy-to-read format:

In [13]:
print(mf)

# Rows: 891
# Columns: 13

Column 1: "PassengerId"
- Variable Type: discrete
- Data Type: Int64
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.uniform
	- Provenance: builtin
	- Parameters:
		- lower: 1
		- upper: 892
	

Column 2: "Name"
- Variable Type: string
- Data Type: Utf8
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.freetext
	- Provenance: builtin
	- Parameters:
		- locale: EN
		- avg_sentences: 2.4691358024691357
		- avg_words: 4.093153759820426
	

Column 3: "Sex"
- Variable Type: categorical
- Data Type: Categorical
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.multinoulli
	- Provenance: builtin
	- Parameters:
		- labels: ['female' 'male']
		- probs: [0.35241302 0.64758698]
	

Column 4: "Age"
- Variable Type: discrete
- Data Type: Int64
- Proportion of Missing Values: 0.1987
- Distribution:
	- Type: core.truncated_normal
	- Provenance: builtin
	- Parameters:
		- lower: -1e-08
		- upper: 80.00000001
		- mean: 2

## Step 3: Exporting the MetaFrame

After creating the MetaFrame, Metasyn can serialize and export it into a GMF file using `mf.export()`, passing in the filepath as a parameter. 


> **GMF files:**
> GMF files are JSON files that follow the [Generative Metadata Format (GMF)](https://github.com/sodascience/generative_metadata_format), a format designed to contain statistical metadata for (tabular) datasets that has been designed to be easy to read and understand. This allows users to audit, understand, modify and share their data generation model with ease.

More information on exporting and importing MetaFrames can be found on the metasyn docs ['exporting and importing metaframes'](https://metasynth.readthedocs.io/en/latest/usage/exporting_metaframes.html) page.

In [14]:
file_path = "example_gmf_titanic.json"

# Serialize and export the MetaFrame to a GMF file
mf.export(file_path)

The GMF file should now be saved to the specified filepath, feel free to open and inspect it!

It's also possible to preview how the exported file would look, without actually saving it to disk. This can be done as follows:

In [15]:
# Get a preview of the GMF file (`repr()`) and print it (`print()`)
print(repr(mf))

{
    "n_rows": 891,
    "n_columns": 13,
    "provenance": {
        "created by": {
            "name": "metasyn",
            "version": "0.7.1.dev1+g1f601ea.d20240226"
        },
        "creation time": "2024-02-27T14:05:22.973480"
    },
    "vars": [
        {
            "name": "PassengerId",
            "type": "discrete",
            "dtype": "Int64",
            "prop_missing": 0.0,
            "distribution": {
                "implements": "core.uniform",
                "version": "1.0",
                "provenance": "builtin",
                "class_name": "DiscreteUniformDistribution",
                "unique": false,
                "parameters": {
                    "lower": 1,
                    "upper": 892
                }
            }
        },
        {
            "name": "Name",
            "type": "string",
            "dtype": "Utf8",
            "prop_missing": 0.0,
            "distribution": {
                "implements": "core.freetext",
          

A GMF file can be imported and loaded into a MetaFrame using the `MetaFrame.from_json()` class method, passing in the file path as a parameter. 

In [16]:
# Create a MetaFrame based on a GMF (.json) file
mf = MetaFrame.from_json(file_path)

## Step 4: Generating synthetic data

Once a MetaFrame is loaded, synthetic data can be generated from it. We can do so by using the the `synthesize` method of the MetaFrame, passing in how many rows the generated data should contain as a parameter. This returns a DataFrame with the synthetic data.

More information on generating synthetic data based on MetaFrames can be found on the metasyn docs, [here](https://metasynth.readthedocs.io/en/latest/usage/generating_synthetic_data.html).

In [17]:
# generate synthetic data
syn_df = mf.synthesize(5)

We can now view the synthetic data:

In [18]:
syn_df

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],f32
112,"""Political. Thu…","""male""",17,0,"""239335""",7.261361,"""A345""","""Q""",1928-04-27,null,2022-07-17 08:29:23,null
435,"""Political. Pho…","""male""",56,0,"""6885""",14.887817,null,"""S""",null,18:36:18,2022-08-05 09:40:30,null
81,"""Fear.""","""male""",14,0,"""34640""",110.997831,null,"""S""",1924-12-05,17:47:37,null,null
630,"""Role within fi…","""male""",17,0,"""MCR/1 0729""",56.553553,"""D94""","""S""",1937-10-26,17:48:46,2022-08-08 11:06:37,null
572,"""Feel. Answer. …","""male""",55,0,"""2641""",55.082475,null,"""C""",1904-05-29,13:34:45,null,null


As you can see, the synthetic data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

The `MetaFrame.fit_dataframe()` method allows you to have more control over how your synthetic dataset is generated by passing in an optional `spec` (short for specification) parameter. `spec` is a dictionary that can be used to give metasyn instructions on a per-variable basis, these instructions can range from setting a variable to be unique, to directly setting its distribution. 

### Spec: Setting variables to have unique variables

During the MetaFrame generation at the start (using `MetaFrame.fit_dataframe()`), metaframe detected a column (PassengerId) as possibly unique, as indicated by the following warning:

> *"Variable PassengerId seems unique, but not set to be unique."*

This is because this column holds a unique identifier for each passenger, which is in fact unique to each passenger. As such, we want the synthetic data generated for this column to be unique as well. 

In order to set a variable to be unique, we can add a `unique` key to the specification dictionary of the variable, and set it to `True`. We can do it for the `PassengerId` column as follows:


In [19]:
# First, we create a specification dictionary for the variables
var_spec = [VarConfig(name="PassengerId", dist_spec=DistributionSpec(unique=True))]

# then, we add that dictionary as the `spec` argument
mf = MetaFrame.fit_dataframe(df, var_specs=var_spec)

# then, let's check what the metadata about PassengerId contains!
mf["PassengerId"].to_dict()

100%|██████████| 13/13 [00:01<00:00,  7.66it/s]


{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'Int64',
 'prop_missing': 0.0,
 'distribution': {'implements': 'core.unique_key',
  'version': '1.0',
  'provenance': 'builtin',
  'class_name': 'UniqueKeyDistribution',
  'unique': True,
  'parameters': {'lower': 1, 'consecutive': True}}}

So let's check what is generated from this new MetaFrame:

In [20]:
mf.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],f32
1,"""Attack.""","""female""",null,0,"""83843""",31.01965,null,"""S""",1937-01-22,18:04:51,2022-07-20 13:40:05,null
2,"""Effect hospita…","""male""",48,0,"""580240""",27.495744,"""F695""","""S""",null,null,2022-07-28 14:59:20,null
3,"""Fire. Write.""","""male""",31,0,"""965067""",32.218029,null,"""S""",1911-12-10,13:04:20,2022-07-31 21:00:09,null
4,"""Husband rememb…","""female""",14,0,"""39638""",94.068273,"""E208""","""S""",1921-11-11,null,2022-07-25 15:29:14,null
5,"""Without glass.…","""female""",54,0,"""32727""",7.836606,null,"""S""",1932-05-06,14:23:40,null,null


As you can see, the `PassengerId` column is now unique!

### Spec: Fake names (and other Faker data types)

Currently, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in metasyn is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, metasyn supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We can specify metasyn to use Faker names for the `Name` column as follows:

In [21]:
# First, we create a specification dictionary for the variables
from metasyn.distribution import FakerDistribution

var_specs = [
    VarConfig(name="PassengerId", dist_spec=DistributionSpec(unique=True)),
    VarConfig(name="Name", dist_spec=FakerDistribution("name")),
]

mf = MetaFrame.fit_dataframe(df, var_specs=var_specs)
mf.synthesize(5)

100%|██████████| 13/13 [00:01<00:00,  9.61it/s]


PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,f32,cat,date,time,datetime[μs],f32
1,"""Lisa Mckenzie""","""male""",null,0,"""0207""",44.652223,null,"""S""",null,13:13:28,2022-07-16 02:22:06,null
2,"""Dennis Curtis""","""male""",39,0,"""850608""",67.276907,null,"""S""",1937-03-27,12:56:27,2022-08-02 05:25:44,null
3,"""George Mahoney…","""female""",22,0,"""53568""",26.153512,null,"""S""",1911-06-29,17:16:35,2022-07-31 11:33:53,null
4,"""Chad Green""","""female""",null,0,"""5634""",67.313575,null,"""C""",1906-12-19,17:33:13,2022-07-18 06:09:45,null
5,"""Peter Lee""","""female""",12,0,"""831644""",52.34483,null,"""S""",1928-05-06,12:29:00,2022-08-03 08:25:10,null


That already looks a lot better for the `Name` column!

### Spec: Setting distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the distribution with the best fit from all available distributions for the variable type. However, we can also manually specify which distribution to fit, or simply specify the distribution including the parameters for the variable.

In [22]:
from metasyn.distribution import DiscreteUniformDistribution

var_specs = [
    VarConfig(name="PassengerId", dist_spec=DistributionSpec(unique=True)),
    VarConfig(name="Name", dist_spec=FakerDistribution("name")),
    VarConfig(name="Name", dist_spec="LogNormalDistribution"), # estimate / fit an exponential distribution based on the data
    VarConfig(name="Age", dist_spec=DiscreteUniformDistribution(20, 40)) # fully specify a distribution for age (uniform between 20 and 40)
]

mf = MetaFrame.fit_dataframe(df, var_specs=var_specs)
mf.synthesize(5)

100%|██████████| 13/13 [00:01<00:00, 10.34it/s]


PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],f32
1,"""Scott Ross""","""male""",36,1,"""450236""",24.084441,"""B4""","""S""",1926-03-19,18:29:11,2022-08-07 09:02:56,null
2,"""Jacob Sims""","""male""",34,0,"""44574""",31.669983,null,"""C""",1934-02-19,17:14:20,2022-07-31 05:32:13,null
3,"""Ronald Fowler""","""male""",31,0,"""165583""",87.990036,"""E9""","""Q""",1935-07-24,11:27:46,2022-08-14 18:17:45,null
4,"""Kristie Smith""","""female""",null,0,"""71957""",74.376833,null,"""S""",1907-09-24,13:14:03,2022-08-10 19:28:33,null
5,"""Jeffrey Moore""","""male""",26,0,"""7794""",21.640565,"""B9""","""C""",1907-01-01,17:12:09,2022-07-22 18:18:53,null


### Spec: Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [23]:
from metasyn.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[A-F][0-9]{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [A-F]?(\d{2,3})?

var_specs = [
    VarConfig(name="PassengerId", dist_spec=DistributionSpec(unique=True)),
    VarConfig(name="Name", dist_spec=FakerDistribution("name")),
    VarConfig(name="Name", dist_spec="LogNormalDistribution"), # estimate / fit an exponential distribution based on the data
    VarConfig(name="Age", dist_spec=DiscreteUniformDistribution(20, 40)), # fully specify a distribution for age (uniform between 20 and 40)
    VarConfig(name="Cabin", dist_spec=cabin_distribution), # Use the regex distribution for the cabin
]

mf = MetaFrame.fit_dataframe(df, var_specs=var_specs)
mf.synthesize(10)

100%|██████████| 13/13 [00:00<00:00, 13.99it/s]


PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],f32
1,"""Rebecca Philli…","""female""",27,0,"""DHVH/I 51386""",3.050459,null,"""S""",1924-11-04,12:35:17,2022-07-26 04:28:30,null
2,"""Kathryn Walker…","""female""",36,0,"""444974""",12.409123,null,"""S""",1918-03-16,13:45:40,2022-08-08 03:25:25,null
3,"""Gregory White""","""female""",34,0,"""662543""",0.4001,"""B12""","""S""",1933-11-09,16:08:40,2022-07-17 07:18:19,null
4,"""Benjamin Schwa…","""female""",37,0,"""9616""",42.126012,null,"""S""",1929-01-20,18:08:28,2022-07-29 10:40:33,null
5,"""Isaiah Morris""","""male""",37,0,"""37804""",62.558723,null,"""Q""",1927-09-07,11:26:25,2022-07-16 13:41:39,null
6,"""Ronald Glass""","""female""",24,0,"""5676""",21.465857,null,"""S""",1907-02-06,16:17:50,2022-07-29 13:35:30,null
7,"""Alvin Ramirez""","""male""",35,0,"""0544""",54.050298,null,"""S""",1937-04-14,12:54:11,null,null
8,"""Mary Nelson""","""female""",23,0,"""2409""",29.474931,null,"""S""",1905-12-26,15:25:06,2022-08-04 01:53:27,null
9,"""Brian Taylor""","""male""",30,0,"""523754""",33.254455,null,"""C""",1928-06-07,18:24:42,null,null


## Step 6: Comparing the final synthetic dataset to the original

Let's first compare the averages of the numerical columns:

In [24]:
df.mean()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
f64,str,cat,f64,f64,str,f64,str,cat,date,time,datetime[μs],str
446.0,null,null,29.693277,0.381594,null,32.204208,null,null,null,null,null,null


In [25]:
mf.synthesize(len(df)).mean()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
f64,str,cat,f64,f64,str,f64,str,cat,date,time,datetime[μs],f32
446.0,null,null,29.925311,0.080808,null,32.871036,null,null,null,null,null,null


Then, we can also see how many missing values are in each column

In [26]:
df.null_count()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,177,0,0,0,687,2,78,79,92,891


In [27]:
mf.synthesize(len(df)).null_count()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,187,0,0,0,673,2,86,83,98,891


## Step 7: Adding descriptions to variables

With the data being taken care of, we can still do one last thing. We can add descriptions to the variables, to clarify what they mean. This can be particularly useful when sharing the `MetaFrame` or generated data with others, as it gives them more context to what they're working with.

One way of adding a description to a variable, is by setting it in the `spec` dictionary, this can be done by simply adding a `description` key with the description as a value. For example, adding a description to the `Cabin` column can be done as follows:

In [28]:
var_specs = [
    # Ensure unique values for the `PassengerId` column
    VarConfig(name="PassengerId", dist_spec=DistributionSpec(unique=True)),

    # Utilize the Faker library to synthesize realistic names for the `Name` column
    VarConfig(name="Name", dist_spec=FakerDistribution("name")),

    # Fit `Fare` to an log-normal distribution, but base the parameters on the data
    VarConfig(name="Name", dist_spec="LogNormalDistribution"),

    # Set the `Age` column to a discrete uniform distribution ranging from 20 to 40
    VarConfig(name="Age", dist_spec=DiscreteUniformDistribution(20, 40)),

    # Use a regex-based distribution to generate `Cabin` values following [A-F][0-9]{2,3}
    VarConfig(name="Cabin", dist_spec=cabin_distribution, description="The cabin number of the passenger."),
]

mf = MetaFrame.fit_dataframe(df, var_specs=var_specs) 

100%|██████████| 13/13 [00:00<00:00, 14.75it/s]


We can get a list of all the descriptions in the fitted `MetaFrame` by accessing its `descriptions` property, as follows:

In [29]:
print(mf.descriptions)

{'Cabin': 'The cabin number of the passenger.'}


Instead of setting the description in the variable specification (which happens before fitting a `MetaFrame` to a `DataFrame`), we can assign a description to an already generated `MetaFrame` by directly setting a column's description attribute. For example, we can assign a description to the `PassengerId` column as follows:

In [30]:
mf["PassengerId"].description = "The ID of each passenger, as assigned by Pandas."

print(mf.descriptions)

{'PassengerId': 'The ID of each passenger, as assigned by Pandas.', 'Cabin': 'The cabin number of the passenger.'}


We can also set multiple descriptions of an already generated `MetaFrame` at once by passing in a dictionary of descriptions to its `descriptions` property. For example, we can set descriptions for the `Age` and `Name` columns as follows:

In [31]:
mf.descriptions = {"Name": "Name of the passenger", "Age": "Age of the passenger in years"}

print(mf.descriptions)

{'PassengerId': 'The ID of each passenger, as assigned by Pandas.', 'Name': 'Name of the passenger', 'Age': 'Age of the passenger in years', 'Cabin': 'The cabin number of the passenger.'}


Instead of a dictionary, it is also possible to pass in a list of descriptions to the `descriptions` property of a `MetaFrame`. 

This can only be done if the list has the same length as the number of variables. In other words, each description must be passed in. 

This can be useful for example when generating placeholder descriptions automatically through list comprehension, as is done in the following example:

In [32]:
mf.descriptions = [f"Placeholder description for {var.name}" for var in mf.meta_vars]

print(mf.descriptions)

{'PassengerId': 'Placeholder description for PassengerId', 'Name': 'Placeholder description for Name', 'Sex': 'Placeholder description for Sex', 'Age': 'Placeholder description for Age', 'Parch': 'Placeholder description for Parch', 'Ticket': 'Placeholder description for Ticket', 'Fare': 'Placeholder description for Fare', 'Cabin': 'Placeholder description for Cabin', 'Embarked': 'Placeholder description for Embarked', 'Birthday': 'Placeholder description for Birthday', 'Board time': 'Placeholder description for Board time', 'Married since': 'Placeholder description for Married since', 'all_NA': 'Placeholder description for all_NA'}


## The end

That's it for this tutorial! You should now have a good understanding of how to use metasyn to generate synthetic data from a dataset. If you want to learn more, check out the [metasyn docs](https://metasynth.readthedocs.io/en/latest/).

If you have any questions, feel free to [reach out](https://metasynth.readthedocs.io/en/latest/about/contact.html).

